# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mizdah,31.4451,12.9801,53.55,50,7,4.88,LY,1708022629
1,1,garden city,42.3256,-83.3310,39.15,87,100,18.41,US,1708022477
2,2,dudinka,69.4058,86.1778,-11.33,95,98,14.00,RU,1708022629
3,3,calipatria,33.1256,-115.5142,63.25,34,100,1.61,US,1708022629
4,4,longyearbyen,78.2186,15.6401,26.44,93,75,9.22,SJ,1708022630


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_p = city_data_df.hvplot.points( "Lng","Lat", geo = True,
    tiles = "OSM", frame_width = 600, frame_height = 400,
    size = "Humidity", scale = 0.5, color = "City"
)

# Display the map
map_p

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[(city_data_df["Max Temp"] >= 30) & (city_data_df["Max Temp"] <= 35),:]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna(how="any")

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
41,41,kitimat,54.0524,-128.6534,33.91,67,0,5.01,CA,1708022635
73,73,rawlins,41.7911,-107.2387,34.36,86,100,17.27,US,1708022642
170,170,kristiansund,63.1115,7.7320,34.45,75,0,4.61,NO,1708022658
199,199,st. john's,47.5649,-52.7093,33.28,98,75,29.93,CA,1708022392
204,204,korla,41.7597,86.1469,32.40,64,85,4.59,CN,1708022375


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = " "

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
41,kitimat,CA,54.0524,-128.6534,67,
73,rawlins,US,41.7911,-107.2387,86,
170,kristiansund,NO,63.1115,7.7320,75,
199,st. john's,CA,47.5649,-52.7093,98,
204,korla,CN,41.7597,86.1469,64,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "limit": "1",
    "apiKey": f"{geoapify_key}"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = hotel_df.loc[index, "Lat"]
    lon = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
kitimat - nearest hotel: No hotel found
rawlins - nearest hotel: No hotel found
kristiansund - nearest hotel: No hotel found
st. john's - nearest hotel: No hotel found
korla - nearest hotel: No hotel found
colby - nearest hotel: No hotel found
dakoutun - nearest hotel: No hotel found
fauske - nearest hotel: No hotel found
corner brook - nearest hotel: No hotel found
selfoss - nearest hotel: No hotel found
laojunmiao - nearest hotel: No hotel found
mount pearl - nearest hotel: No hotel found
malmslatt - nearest hotel: No hotel found
pingliang - nearest hotel: No hotel found
pernio - nearest hotel: No hotel found
miquelon - nearest hotel: No hotel found
sinan - nearest hotel: No hotel found
manistee - nearest hotel: No hotel found
gananoque - nearest hotel: No hotel found
conception bay south - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
41,kitimat,CA,54.0524,-128.6534,67,No hotel found
73,rawlins,US,41.7911,-107.2387,86,No hotel found
170,kristiansund,NO,63.1115,7.7320,75,No hotel found
199,st. john's,CA,47.5649,-52.7093,98,No hotel found
204,korla,CN,41.7597,86.1469,64,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_df = hotel_df.drop(hotel_df[hotel_df["Hotel Name"] == "No hotel found"].index)
map_p = hotel_df.hvplot.points( "Lng","Lat", geo = True,
    tiles = "OSM", frame_width = 600, frame_height = 400,
    size = "Humidity", scale = 0.5, color = "City",
    hover_cols = ["Hotel Name", "Country"]
)


# Display the map
map_p

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)